In [ ]:
import os
# os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.7/bin") # path to nvidia dlls, must have CUDA drivers installed
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision

### GPU Acceleration

** IMPORTANT NOTE **
 - You must have CUDA driver and CUDA toolkit installed on your machine. To do so, follow the instructions here:
https://www.tensorflow.org/install/pip

- Enable GPU Acceleration if you have a NVIDIA GPU with compute capability >= 7.0

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

# ## ENABLE THIS IF YOU HAVE RTX GPU WITH COMPUTE CAPABILITY 7.0 or higher
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

### Loading Data

In [ ]:
fp = "../data/features_combined.csv"
batch_pd = pd.read_csv(fp, index_col=False)
dataset = batch_pd.copy()
dataset.sort_values(by=['policy'], ascending=True, inplace=True)

# dataset
dataset.isna().sum()
dataset = dataset.dropna().drop(columns=['policy', 'barcode'])

### Test run setup
1. Adjust global model config to test the model settings
2. Change the Run configuration to define the saved location for the results and the number of runs
3. Run the notebook
4. Clear output and restart the notebook for the next run.

In [ ]:
## Global Model Config
EPOCHS = 2500
UNITS = 1
LEARNING_RATE = 0.01
CALLBACK = tf.keras.callbacks.EarlyStopping(monitor='RMSE', patience=15, min_delta=0.01)

#### Run configuration

In [ ]:
## Define the run setup

RUN_NUMBER = 1 # number of runs
SAVED_RESULT_PATH = "../variance_model/results/test_run#" + str(RUN_NUMBER) # path to saved results

if ( os.path.exists(SAVED_RESULT_PATH) ):
    print("Directory " , SAVED_RESULT_PATH , " already exists")
else:
    os.mkdir(SAVED_RESULT_PATH)
    print("Directory " , SAVED_RESULT_PATH , " created")

## Data split
Split by policy fast charge first(5C - 8C), and then by policy slow charge (1C - 4C)


In [ ]:
fast_charge_dataset = dataset.iloc[29:, :] 
print(fast_charge_dataset.shape) # 51 data points 

## Fast-Charge Test-Train split
Selecting alternate batteries for training and testing

In [ ]:
fast_charge_train_ds = fast_charge_dataset.iloc[0::2, :]
fast_charge_test_ds = fast_charge_dataset.iloc[1::2, :]

fastcharge_train_features = fast_charge_train_ds.copy()
fastcharge_test_features = fast_charge_test_ds.copy()

train_labels = fastcharge_train_features.pop('cycle_life')
test_labels = fastcharge_test_features.pop('cycle_life')

# Linear Regress
### Layering and Build Model

In [ ]:
# Normalize layer
QDiffLinVar = np.array(fastcharge_train_features['QDiffLinVar'])
QDiffLinVar_normalizer = layers.Normalization(input_shape=[1,], axis=None)
QDiffLinVar_normalizer.adapt(QDiffLinVar)

# Input and output layers
variance_fastcharge_basemodel = tf.keras.Sequential([
    QDiffLinVar_normalizer,
    layers.Dense(UNITS, input_dim=1 ,activation='relu' ),
    layers.Dense(1, activation='linear', dtype='float32', name='predictions')
])

## Compile the model and make a prediction
variance_fastcharge_basemodel.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[ 
        tf.keras.metrics.RootMeanSquaredError( name='RMSE'), 
        'mae']
    )

variance_fastcharge_basemodel.summary()

print("Number of weights:", len(variance_fastcharge_basemodel.weights))
print("trainable_weights:", len(variance_fastcharge_basemodel.trainable_weights))
print("non_trainable_weights:", len(variance_fastcharge_basemodel.non_trainable_weights))

### Train the Model

In [ ]:
%%time
history = variance_fastcharge_basemodel.fit(
    fastcharge_train_features['QDiffLinVar'],
    train_labels,
    epochs=EPOCHS,
    verbose=2,
    # callbacks=[CALLBACK],
    validation_data=(fastcharge_test_features['QDiffLinVar'], test_labels)
    )

### Plot loss graph and Evaluate

In [ ]:
def plot_loss(history):
  plt.figure("Fast-charge base model loss", figsize=(5,5), dpi=100, facecolor='w', edgecolor='k')
  plt.plot(history['RMSE'], label='loss', color='magenta')
  plt.plot(history['val_RMSE'], label='val_loss', color='teal', linestyle='--')
  plt.ylim([0, 800])
  plt.xlabel('Epoch')
  plt.ylabel('Error [cycles]')
  plt.legend()
  plt.grid(True)
  plt.title('Fast-charge base model loss')
  plt.savefig(SAVED_RESULT_PATH + "/fastcharge_basemodel_loss_RUN#" + str(RUN_NUMBER) + ".png")
  

## Display model's loss and accuracy history
fastcharge_basemodel_history = pd.DataFrame(history.history)
fastcharge_basemodel_history.to_csv(SAVED_RESULT_PATH + "/fastcharge_basemodel_history_RUN#" + str(RUN_NUMBER) + ".csv")
plot_loss(fastcharge_basemodel_history)

In [ ]:
test_results = {}
test_results['variance_fastcharge_basemodel'] = variance_fastcharge_basemodel.evaluate(
    fastcharge_test_features['QDiffLinVar'],
    test_labels, verbose=1) #sqrt for mse

### Make Predictions

In [ ]:
def plot_prediction(y_train, y_test):
  plt.figure("Fast-charge Base-model Prediction", figsize=(5,5), dpi=100, facecolor='w', edgecolor='k')
  plt.axes(aspect='equal')
  plt.scatter(y_train, train_labels, label='Predictions (train)')
  plt.scatter(y_test, test_labels, label='Predictions (test)')
  lims = [0, 2000]
  plt.xlim(lims)
  plt.ylim(lims)
  plt.plot(lims, lims, 'k', )
  plt.xlabel('Predicted Cycle life')
  plt.ylabel('Actual Cycle life')
  plt.legend()
  plt.title('Fast Charge Model Prediction')
  plt.savefig( SAVED_RESULT_PATH + '/fastcharge_basemodel_prediction_RUN#' + str(RUN_NUMBER) + '.png')

train_prediction = variance_fastcharge_basemodel.predict(fast_charge_train_ds['QDiffLinVar'])
test_prediction = variance_fastcharge_basemodel.predict(fast_charge_test_ds['QDiffLinVar'])
plot_prediction(train_prediction, test_prediction)

### Final Results

In [ ]:

result = pd.DataFrame(test_results, index=['MSE', 'RMSE', 'MAE']).T
result.to_csv( SAVED_RESULT_PATH + '/fastcharge_basemodel_result_RUN#' + str(RUN_NUMBER) + ".csv")

### Saving the model

In [ ]:
variance_fastcharge_basemodel.save('../variance_model/saved_model/variance_fastcharge_basemodel')